In [1]:
from itertools import product
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# ─────────────────────────────────────────────────────────────────────────────
#  Define all 45 canonical graphs with their edge‐signs and gate lists.
#  Each dict has:
#    • "id"      : integer 0..44
#    • "edges"   : {"eAA": "+"/"-"/"0", "eAB": ..., "eBA": ..., "eBB": ...}
#    • "gates_A" : list of gate‐strings for node A (e.g. "(A)&(B)", "(A)", "(0)", "(!A)")
#    • "gates_B" : same for node B
# ─────────────────────────────────────────────────────────────────────────────

graphs = [
    {"id": 44, "edges": {"eAA": "+", "eAB": "+", "eBA": "+", "eBB": "+"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(A)&(B)", "(A)|(B)"]},
    {"id": 43, "edges": {"eAA": "+", "eAB": "+", "eBA": "+", "eBB": "-"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 42, "edges": {"eAA": "+", "eAB": "+", "eBA": "+", "eBB": "0"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(A)"]},
    {"id": 41, "edges": {"eAA": "+", "eAB": "+", "eBA": "-", "eBB": "+"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(A)&(B)", "(A)|(B)"]},
    {"id": 40, "edges": {"eAA": "+", "eAB": "+", "eBA": "-", "eBB": "-"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 39, "edges": {"eAA": "+", "eAB": "+", "eBA": "-", "eBB": "0"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(A)"]},
    {"id": 38, "edges": {"eAA": "+", "eAB": "+", "eBA": "0", "eBB": "+"},
     "gates_A": ["(A)"],                   "gates_B": ["(A)&(B)", "(A)|(B)"]},
    {"id": 37, "edges": {"eAA": "+", "eAB": "+", "eBA": "0", "eBB": "-"},
     "gates_A": ["(A)"],                   "gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 36, "edges": {"eAA": "+", "eAB": "+", "eBA": "0", "eBB": "0"},
     "gates_A": ["(A)"],                   "gates_B": ["(A)"]},
    {"id": 35, "edges": {"eAA": "+", "eAB": "-", "eBA": "+", "eBB": "-"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(!A)&(!B)", "(!A)|(!B)"]},
    {"id": 34, "edges": {"eAA": "+", "eAB": "-", "eBA": "+", "eBB": "0"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(!A)"]},
    {"id": 33, "edges": {"eAA": "+", "eAB": "-", "eBA": "-", "eBB": "+"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(!A)&(B)", "(!A)|(B)"]},
    {"id": 32, "edges": {"eAA": "+", "eAB": "-", "eBA": "-", "eBB": "-"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(!A)&(!B)", "(!A)|(!B)"]},
    {"id": 31, "edges": {"eAA": "+", "eAB": "-", "eBA": "-", "eBB": "0"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(!A)"]},
    {"id": 30, "edges": {"eAA": "+", "eAB": "-", "eBA": "0", "eBB": "+"},
     "gates_A": ["(A)"],                   "gates_B": ["(!A)&(B)", "(!A)|(B)"]},
    {"id": 29, "edges": {"eAA": "+", "eAB": "-", "eBA": "0", "eBB": "-"},
     "gates_A": ["(A)"],                   "gates_B": ["(!A)&(!B)", "(!A)|(!B)"]},
    {"id": 28, "edges": {"eAA": "+", "eAB": "-", "eBA": "0", "eBB": "0"},
     "gates_A": ["(A)"],                   "gates_B": ["(!A)"]},
    {"id": 27, "edges": {"eAA": "+", "eAB": "0", "eBA": "+", "eBB": "-"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(!B)"]},
    {"id": 26, "edges": {"eAA": "+", "eAB": "0", "eBA": "+", "eBB": "0"},
     "gates_A": ["(A)&(B)", "(A)|(B)"],    "gates_B": ["(0)"]},
    {"id": 25, "edges": {"eAA": "+", "eAB": "0", "eBA": "-", "eBB": "-"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(!B)"]},
    {"id": 24, "edges": {"eAA": "+", "eAB": "0", "eBA": "-", "eBB": "0"},
     "gates_A": ["(A)&(!B)", "(A)|(!B)"],  "gates_B": ["(0)"]},
    {"id": 23, "edges": {"eAA": "+", "eAB": "0", "eBA": "0", "eBB": "+"},
     "gates_A": ["(A)"],                   "gates_B": ["(B)"]},
    {"id": 22, "edges": {"eAA": "+", "eAB": "0", "eBA": "0", "eBB": "-"},
     "gates_A": ["(A)"],                   "gates_B": ["(!B)"]},
    {"id": 21, "edges": {"eAA": "0", "eAB": "+", "eBA": "0", "eBB": "0"},
     "gates_A": ["(0)"],                   "gates_B": ["(A)"]},
    {"id": 20, "edges": {"eAA": "0", "eAB": "+", "eBA": "-", "eBB": "0"},
     "gates_A": ["(!B)"],                  "gates_B": ["(A)"]},
    {"id": 19, "edges": {"eAA": "0", "eAB": "+", "eBA": "+", "eBB": "0"},
     "gates_A": ["(B)"],                   "gates_B": ["(A)"]},
    {"id": 18, "edges": {"eAA": "0", "eAB": "0", "eBA": "0", "eBB": "0"},
     "gates_A": ["(0)"],                   "gates_B": ["(0)"]},
    {"id": 17, "edges": {"eAA": "0", "eAB": "-", "eBA": "0", "eBB": "0"},
     "gates_A": ["(0)"],                   "gates_B": ["(!A)"]},
    {"id": 16, "edges": {"eAA": "0", "eAB": "-", "eBA": "-", "eBB": "0"},
     "gates_A": ["(!B)"],                  "gates_B": ["(!A)"]},
    {"id": 15, "edges": {"eAA": "-", "eAB": "0", "eBA": "0", "eBB": "0"},
     "gates_A": ["(!A)"],                  "gates_B": ["(0)"]},
    {"id": 14, "edges": {"eAA": "-", "eAB": "0", "eBA": "0", "eBB": "-"},
     "gates_A": ["(!A)"],                  "gates_B": ["(!B)"]},
    {"id": 13, "edges": {"eAA": "-", "eAB": "0", "eBA": "-", "eBB": "0"},
     "gates_A": ["(!A)&(!B)", "(!A)|(!B)"],"gates_B": ["(0)"]},
    {"id": 12, "edges": {"eAA": "-", "eAB": "0", "eBA": "+", "eBB": "0"},
     "gates_A": ["(!A)&(B)", "(!A)|(B)"],  "gates_B": ["(0)"]},
    {"id": 11, "edges": {"eAA": "-", "eAB": "-", "eBA": "0", "eBB": "0"},
     "gates_A": ["(!A)"],                  "gates_B": ["(!A)"]},
    {"id": 10, "edges": {"eAA": "-", "eAB": "-", "eBA": "-", "eBB": "0"},
     "gates_A": ["(!A)&(!B)", "(!A)|(!B)"],"gates_B": ["(!A)"]},
    {"id": 9,  "edges": {"eAA": "-", "eAB": "-", "eBA": "-", "eBB": "-"},
     "gates_A": ["(!A)&(!B)", "(!A)|(!B)"],"gates_B": ["(!A)&(!B)", "(!A)|(!B)"]},
    {"id": 8,  "edges": {"eAA": "-", "eAB": "-", "eBA": "+", "eBB": "0"},
     "gates_A": ["(!A)&(B)", "(!A)|(B)"],  "gates_B": ["(!A)"]},
    {"id": 7,  "edges": {"eAA": "-", "eAB": "+", "eBA": "0", "eBB": "0"},
     "gates_A": ["(!A)"],                  "gates_B": ["(A)"]},
    {"id": 6,  "edges": {"eAA": "-", "eAB": "+", "eBA": "0", "eBB": "-"},
     "gates_A": ["(!A)"],                  "gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 5,  "edges": {"eAA": "-", "eAB": "+", "eBA": "-", "eBB": "0"},
     "gates_A": ["(!A)&(!B)", "(!A)|(!B)"],"gates_B": ["(A)"]},
    {"id": 4,  "edges": {"eAA": "-", "eAB": "+", "eBA": "-", "eBB": "-"},
     "gates_A": ["(!A)&(!B)", "(!A)|(!B)"],"gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 3,  "edges": {"eAA": "-", "eAB": "+", "eBA": "+", "eBB": "0"},
     "gates_A": ["(!A)&(B)", "(!A)|(B)"],  "gates_B": ["(A)"]},
    {"id": 2,  "edges": {"eAA": "-", "eAB": "+", "eBA": "+", "eBB": "-"},
     "gates_A": ["(!A)&(B)", "(!A)|(B)"],  "gates_B": ["(A)&(!B)", "(A)|(!B)"]},
    {"id": 1,  "edges": {"eAA": "+", "eAB": "0", "eBA": "0", "eBB": "0"},
     "gates_A": ["(A)"],                   "gates_B": ["(0)"]},
    {"id": 0,  "edges": {"eAA": "+", "eAB": "0", "eBA": "0", "eBB": "+"},
     "gates_A": ["(A)"],                   "gates_B": ["(B)"]}
]

# ─────────────────────────────────────────────────────────────────────────────
#  Create output directory if it doesn’t already exist
# ─────────────────────────────────────────────────────────────────────────────
output_dir = Path("stgs_sync_manual")
output_dir.mkdir(exist_ok=True)


# ─────────────────────────────────────────────────────────────────────────────
#  Evaluate a single gate string "(...)" at a given Boolean state.
#  Input:
#    gate:   string like "(A)&(B)", "(A)", "(!A)", "(0)", "(A)|(!B)", etc.
#    a_val:  integer 0 or 1, current value of A
#    b_val:  integer 0 or 1, current value of B
#  Returns:
#    next_val: 0 or 1, result of evaluating the gate under a_val, b_val
# ─────────────────────────────────────────────────────────────────────────────
def eval_gate(gate: str, a_val: int, b_val: int) -> int:
    # Strip whitespace and outer parentheses
    s = gate.replace(" ", "").lstrip("(").rstrip(")")

    # Constant 0 or 1
    if s == "0":
        return 0
    if s == "1":
        return 1

    # Single literal
    if s == "A":
        return a_val
    if s == "B":
        return b_val
    if s.startswith("!"):
        var = s[1:]
        if var == "A":
            return 1 - a_val
        elif var == "B":
            return 1 - b_val

    # Two‐input AND
    if "&" in s:
        parts = s.split("&")
        result = 1
        for part in parts:
            if part == "A":
                result &= a_val
            elif part == "B":
                result &= b_val
            elif part == "!A":
                result &= (1 - a_val)
            elif part == "!B":
                result &= (1 - b_val)
        return result

    # Two‐input OR
    if "|" in s:
        parts = s.split("|")
        result = 0
        for part in parts:
            if part == "A":
                result |= a_val
            elif part == "B":
                result |= b_val
            elif part == "!A":
                result |= (1 - a_val)
            elif part == "!B":
                result |= (1 - b_val)
        return result

    raise ValueError(f"Cannot parse/evaluate gate '{gate}'.")


# ─────────────────────────────────────────────────────────────────────────────
#  Draw a two‐node signed graph (A, B) with:
#    • green solid arrows for activation (“+”)
#    • red dashed arrows for inhibition (“–”)
#    • no edge if “0”
# ─────────────────────────────────────────────────────────────────────────────
def draw_signed_graph(edges: dict, ax):
    """
    edges: {"eAA": "+"/"-"/"0", "eAB": ..., "eBA": ..., "eBB": ...}
    """
    G = nx.DiGraph()
    G.add_nodes_from(["A", "B"])
    pos = {"A": (0, 0), "B": (1, 0)}

    activating = []
    inhibiting = []
    if edges["eAA"] == "+":
        activating.append(("A", "A"))
    elif edges["eAA"] == "-":
        inhibiting.append(("A", "A"))
    if edges["eAB"] == "+":
        activating.append(("A", "B"))
    elif edges["eAB"] == "-":
        inhibiting.append(("A", "B"))
    if edges["eBA"] == "+":
        activating.append(("B", "A"))
    elif edges["eBA"] == "-":
        inhibiting.append(("B", "A"))
    if edges["eBB"] == "+":
        activating.append(("B", "B"))
    elif edges["eBB"] == "-":
        inhibiting.append(("B", "B"))

    G.add_edges_from(activating + inhibiting)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos,
                           node_color="lightblue",
                           node_size=500,
                           ax=ax)
    nx.draw_networkx_labels(G, pos,
                            font_size=12,
                            font_weight="bold",
                            ax=ax)

    # Draw activation edges in green solid
    for edge in activating:
        style = {}
        if edge[0] == edge[1]:
            style["connectionstyle"] = "arc3,rad=0.5"
        elif (edge[1], edge[0]) in activating or (edge[1], edge[0]) in inhibiting:
            style["connectionstyle"] = "arc3,rad=0.3"
        else:
            style["connectionstyle"] = "arc3,rad=0"
        nx.draw_networkx_edges(G, pos,
                               edgelist=[edge],
                               edge_color="green",
                               arrows=True,
                               arrowstyle="->",
                               arrowsize=15,
                               **style,
                               ax=ax)

    # Draw inhibition edges in red dashed
    for edge in inhibiting:
        style = {}
        if edge[0] == edge[1]:
            style["connectionstyle"] = "arc3,rad=0.5"
        elif (edge[1], edge[0]) in activating or (edge[1], edge[0]) in inhibiting:
            style["connectionstyle"] = "arc3,rad=-0.3"
        else:
            style["connectionstyle"] = "arc3,rad=0"
        nx.draw_networkx_edges(G, pos,
                               edgelist=[edge],
                               edge_color="red",
                               style="dashed",
                               arrows=True,
                               arrowstyle="->",
                               arrowsize=15,
                               **style,
                               ax=ax)

    ax.axis("off")


# ─────────────────────────────────────────────────────────────────────────────
#  Write a small networkx.DiGraph of four nodes ("00","01","10","11") to DOT text.
#  We simply emit:
#    digraph {
#      "00";
#      "01";
#      "10";
#      "11";
#      "00" -> "10";
#      ...
#    }
# ─────────────────────────────────────────────────────────────────────────────
def write_dot_manually(stg: nx.DiGraph, path: Path):
    with open(path, "w") as f:
        f.write("digraph {\n")
        # Declare each of the four nodes (in sorted order)
        for node in sorted(stg.nodes()):
            f.write(f'  "{node}";\n')
        # Write every directed edge
        for u, v in stg.edges():
            f.write(f'  "{u}" -> "{v}";\n')
        f.write("}\n")


# ─────────────────────────────────────────────────────────────────────────────
#  Generate the “manual” synchronous STG for one graph×gate combo:
#    • Panel 1: draw the signed graph
#    • Panel 2: build 4‐node stg and draw it
#    • Panel 3: list gate definitions
#  Save a DOT string and a PDF of the 3‐panel figure.
# ─────────────────────────────────────────────────────────────────────────────
def generate_stg_manual(graph_id: int,
                        edges: dict,
                        gate_a: str,
                        gate_b: str,
                        output_dir: Path):
    """
    graph_id:  int 0..44
    edges:     { "eAA": "+"/"-"/"0", "eAB":..., "eBA":..., "eBB":... }
    gate_a:    string e.g. "(A)&(B)", "(A)", "(0)", "(A)|(!B)", etc.
    gate_b:    same for node B
    """
    # Build a 4‐node directed graph on "00","01","10","11"
    stg = nx.DiGraph()
    all_states = ["00", "01", "10", "11"]
    stg.add_nodes_from(all_states)

    # For each state "ab", compute next_A, next_B
    for state in all_states:
        a_val = int(state[0])
        b_val = int(state[1])

        next_A = eval_gate(gate_a, a_val, b_val)
        next_B = eval_gate(gate_b, a_val, b_val)
        successor = f"{next_A}{next_B}"
        stg.add_edge(state, successor)

    # Create a Matplotlib figure with 3 panels
    fig = plt.figure(figsize=(12, 6))
    gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 1])

    # ── Panel 1: Signed graph ─────────────────────────────────────────────────
    ax1 = fig.add_subplot(gs[0])
    ax1.set_title(
        f"Graph {graph_id:02d}\n"
        f" eAA={edges['eAA']}, eAB={edges['eAB']}\n"
        f" eBA={edges['eBA']}, eBB={edges['eBB']}",
        fontsize=10
    )
    draw_signed_graph(edges, ax1)

    # ── Panel 2: draw the synchronous STG ──────────────────────────────────────
    ax2 = fig.add_subplot(gs[1])
    ax2.set_title(f"Synchronous STG: A={gate_a}, B={gate_b}", fontsize=10)
    pos_stg = nx.spring_layout(stg, seed=42)
    nx.draw_networkx_nodes(stg, pos_stg,
                           node_color="lightgray",
                           node_size=300,
                           ax=ax2)
    nx.draw_networkx_labels(stg, pos_stg,
                            font_size=8,
                            ax=ax2)
    nx.draw_networkx_edges(stg, pos_stg,
                           edge_color="black",
                           arrows=True,
                           arrowstyle="->",
                           arrowsize=10,
                           ax=ax2)
    ax2.axis("off")

    # ── Panel 3: list the gate definitions ──────────────────────────────────────
    ax3 = fig.add_subplot(gs[2])
    ax3.axis("off")
    gates_text = f"Gates:\n  A: {gate_a}\n  B: {gate_b}"
    ax3.text(0.0, 0.5, gates_text,
             fontsize=10, fontfamily="monospace", va="center")

    # 4) Save DOT and PDF
    safe_A = (gate_a
              .replace("&", "and")
              .replace("|", "or")
              .replace("(", "")
              .replace(")", "")
              .replace("!", "not"))
    safe_B = (gate_b
              .replace("&", "and")
              .replace("|", "or")
              .replace("(", "")
              .replace(")", "")
              .replace("!", "not"))

    dot_filename = output_dir / f"graph_{graph_id:02d}_A_{safe_A}_B_{safe_B}.dot"
    pdf_filename = output_dir / f"graph_{graph_id:02d}_A_{safe_A}_B_{safe_B}.pdf"

    write_dot_manually(stg, dot_filename)

    with PdfPages(pdf_filename) as pdf:
        pdf.savefig(fig, bbox_inches="tight")
    plt.close(fig)


# ─────────────────────────────────────────────────────────────────────────────
#  Main loop: for each graph (0..44) and each (gateA, gateB) combination,
#  build and save the manual synchronous STG.
# ─────────────────────────────────────────────────────────────────────────────
total_stgs = 0
for graph in graphs:
    gid = graph["id"]
    edges = graph["edges"]
    for gateA, gateB in product(graph["gates_A"], graph["gates_B"]):
        generate_stg_manual(gid, edges, gateA, gateB, output_dir)
        total_stgs += 1

print(f"✅ Done — generated {total_stgs} synchronous STGs in '{output_dir}'")
print(f"Total graphs: {len(graphs)}   Total STGs: {total_stgs}")

✅ Done — generated 94 synchronous STGs in 'stgs_sync_manual'
Total graphs: 45   Total STGs: 94
